<a href="https://colab.research.google.com/github/Lahiru-mta/Landsat-averaged-image-downloader/blob/main/LandSat_averaged_data_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Get authorize access by Earth Engine
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UWdzoGetG6uVCfPwEYly9405rEQNKmRYhDAxpk14F7o&tc=C-nExu5FsEaIClbZXg5peNHOAwetP8Obe03In7pz6eE&cc=_rz3usZwTMs4UZyt_HqPEd04TIegxB7U0D059SPMWQE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJp5GG5WX-FaGREzSXRg_6kRS6VXg-tj42JWEQsxSA-L6lKrQ0CuZiY

Successfully saved authorization token.


In [2]:
#@title Connect to Google Drive
#@markdown (To save outputs in the drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Import dependencies
from IPython.display import clear_output 

!pip install geemap --quiet
clear_output()
!pip install pydub --quiet
clear_output()
!pip install geopandas --quiet
clear_output()

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
from IPython.display import clear_output 
from IPython.utils import io
import ipywidgets as widgets
%matplotlib inline
from datetime import datetime, timedelta
import gdown
import geemap
import time

from tqdm.notebook import tqdm

import shutil
import os
import zipfile
from pydub import AudioSegment
from pydub.utils import make_chunks

#for raster manupulation
!pip install rasterio --quiet
import rasterio
from rasterio.merge import merge
import glob
from rasterio.plot import show

clear_output()

In [4]:
#@title Functions

def tometers(degrees):
  return(degrees * 111139)

def todegrees(meters):
  return(meters / 111139)

def rectangle(lon_1, lon_2, lat_1, lat_2):
    AOI_coordinates =  [[lon_1, lat_2],
                        [lon_1, lat_1],
                        [lon_2, lat_1],
                        [lon_2, lat_2],
                        [lon_1, lat_2]]

    aoi = ee.Geometry.Polygon(AOI_coordinates)
    return aoi


In [11]:
#@title Import the shape file of the study area
#@markdown paste the Google Drive Shared link of the Zip below ex: " https://drive.google.com/file/d/1QKimP8DQl7cgHJ8SpYitwTOwvPGdu-lt/view?usp=sharing "
ZIP_link = 'https://drive.google.com/file/d/1M64mxw-TvHfIkdAlmdYx7_joFpaTH4a6/view?usp=sharing'#@param{type : "string"}
# 'https://drive.google.com/uc?id=1P3EFH5v8pOkJOUbDEmB_EpmqIbrHUZHk'
url = 'https://drive.google.com/uc?id=' + ZIP_link[32:65]
File_name = 'field_data' #param{type : "string"}
#@markdown *  ***please make sure the shape file is in WGS84 projection***
out = File_name +'.zip'

d_file_name = gdown.download(url, out, quiet=True)

zip_file = '/content/'+out
path = '/content/'+ File_name
zip_ref = zipfile.ZipFile(zip_file)
try:
    os.mkdir(path)
except:
    print('')
zip_ref.extractall(path)
zip_ref.close()

arr = os.listdir(path)
arr2=[]
for i in range(len(arr)):
  if arr[i].endswith('shp'):
    arr2.append(arr[i])
# print(arr2)

if len(arr2)>1:

  print(d_file_name+' contains '+str(len(arr2))+' shape files\n'
          + 'Select the required shape file below')

  shape_file = widgets.Dropdown(    options=arr2,    value=arr2[0],    description='Shape File:')
  display(shape_file)
else:
  shape_file = arr2[0]

# https://github.com/giswqs/geemap/blob/master/examples/notebooks/10_shapefiles.ipynb
# Refer the following code as well
# https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb


In [12]:
#@title Parameters
#@markdown Parameters for the landsat 8 image collection
Start_Date = '2019-01-01' #@param {type:"date"}
start_date = Start_Date
End_Date = '2020-01-01'  #@param {type:"date"}
end_date = End_Date
Scale = 30  #@param {type : 'slider', min:0, max:500, step:10  }
scale = Scale


#@title Create an AOI

#@title Create an AOI
if len(arr2)>1:
  study_shp = path +'/'+ shape_file.value
else:
  study_shp = path +'/'+ shape_file
study = geemap.shp_to_ee(study_shp)


padding = 500 #@param {type : 'slider', min:0, max:10000, step:100  }
padding_d = todegrees(padding)
bound_coordinates = study.geometry().bounds().coordinates().getInfo()[0]
Upper_left = [a - padding_d for a in bound_coordinates[0]]
Bottom_right = [b + padding_d for b in bound_coordinates[2]]

lon_1 = Upper_left[0]
lon_2 = Bottom_right[0]
lat_1 = Upper_left[1]
lat_2 = Bottom_right[1]

AOI_coordinates =  [[lon_1, lat_2],
                    [lon_1, lat_1],
                    [lon_2, lat_1],
                    [lon_2, lat_2],
                    [lon_1, lat_2]]
      

#@title Create tiles
Max_tile_size_km = 15 #@param {type : 'slider', min:0, max:50, step:5  }
tile_size = todegrees(Max_tile_size_km * 1000)


Show_on_Map = False #@param {type:"boolean"}
# coords = geoJason['features'][0]['geometry']['coordinates']
# aoi = ee.Geometry.Polygon(coords)
aoi = ee.Geometry.Polygon(AOI_coordinates)

center = [(lat_1+lat_2)/2,(lon_1+lon_2)/2]
if Show_on_Map == True:
  Map = geemap.Map(center = (center[0], center[1]), zoom=9)
  Map.addLayer(aoi,{},'aoi')
  Map.addLayer(study, {}, 'study area')
  display(Map)



In [27]:
#@title Collect Images


satellite = 'Landsat 8' #@param ['Landsat 9', 'Landsat 8','Landsat 7','Landsat 5','Landsat 4']

tier = 2 #@param {type : 'slider', min:1, max:2, step:1  }
level = 2 #@param {type : 'slider', min:1, max:2, step:1  }

stat_method = 'median' #@param ['median','mean','mode']

image_collections =  {'Landsat 9': "LANDSAT/LC09/C02/T%s_L%s" %(tier,level),
                      'Landsat 8': "LANDSAT/LC08/C02/T%s_L%s" %(tier,level),
                      'Landsat 7': "LANDSAT/LE07/C02/T%s_L%s" %(tier,level),
                      'Landsat 5': "LANDSAT/LT05/C02/T%s_L%s" %(tier,level),
                      'Landsat 4': "LANDSAT/LT04/C02/T%s_L%s" %(tier,level)}

selected_img_col = image_collections[satellite]

url_extension = selected_img_col.replace("/", "_" )

print('For more details about the Image collection, visit:')

print('https://developers.google.com/earth-engine/datasets/catalog/'+url_extension)

img_col = (ee.ImageCollection(selected_img_col)
          .filterBounds(study)
          .filterDate(start_date,end_date)
          .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
          .sort('date'))

if stat_method == 'median':
  result = img_col.median()

elif stat_method == 'mean':
  result = img_col.mean()

elif stat_method == 'mode':
  result = img_col.mode()

For more details about the Image collection, visit:
https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T2_L2


In [29]:
#@title Export


n_cols = int((lon_2 - lon_1) / tile_size) + 1
n_rows = int((lat_2 - lat_1) / tile_size) + 1
n_tiles = n_cols * n_rows
n_tiles
e_0 = lon_1
e = [e_0]
for aa in range (n_cols):
  globals()['e_%s' % str(aa+1)] = e_0 + (lon_2 - lon_1)/n_cols * (aa+1)
  e.append(globals()['e_%s' % str(aa+1)])
# e
n_0 = lat_1
n = [n_0]
for ab in range (n_rows):
  globals()['e_%s' % str(ab+1)] = n_0 + (lat_2 - lat_1)/n_rows * (ab+1)
  n.append(globals()['e_%s' % str(ab+1)])
# n
ac = 0
aois = []
for ae in range (len(e)-1):
  for an in range (len(n)-1):
      globals()['aoi_%s' % ac] = rectangle(e[ae],e[ae+1],n[an],n[an+1])
      aois.append('aoi_' + str(ac))
      ac = ac +1

print ('\n',len(aois), 'tiles created')

#@title Export Data 
Export_to = '/content/drive/MyDrive/geeethya_para_3' #@param{type : "string"}
folder = Export_to + '/tiles_%s' %start_date

created_path = os.path.isdir(folder)
if created_path == True:
    print('Folder already exists')

else:

  try:
      os.mkdir(Export_to)
  except:
      print('path not created')

  try:
      os.mkdir(folder)
  except:
      print('folder not created')

created_path = os.path.isdir(folder)
if created_path == False:
    print('Something went wrong please create the path manualy and come back')


for roi in aois:
  c_aoi = globals()[roi]
  path = folder+'/'+start_date+'_'+roi+'.tif'
  # with io.capture_output() as captured:
  # print(path)
  geemap.ee_export_image(result, filename=path, scale=scale, region=c_aoi, file_per_band=False)


print ('%s TIFF files were saved at %s/' %(start_date,folder))

path2 = Export_to+'/mosaic_%s.tif' %start_date
search_criteria = "*aoi*.tif"
q = os.path.join(folder, search_criteria)
dem_fps = glob.glob(q)
src_files_to_mosaic = []
for fp in dem_fps:
  src = rasterio.open(fp)
  src_files_to_mosaic.append(src)
mosaic, out_trans = merge(src_files_to_mosaic)

out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
              "height": mosaic.shape[1],
              "width": mosaic.shape[2],
              "transform": out_trans,
              }
              )

with rasterio.open(path2, "w", **out_meta) as dest:
  dest.write(mosaic)

print ('\r','Result was saved at %s ' %path2 , end = '')


 6 tiles created
path not created
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_3/tiles_2019-01-01/2019-01-01_aoi_0.tif
Generating URL ...


KeyboardInterrupt: ignored